### Import modules

In [1]:
import simpy
import numpy as np
import numpy.random as random

### Parameters

In [2]:
LAMBDA = 5.0
MU = 8.0
c = 2
MAXSIMTIME = 10000
POPULATION = 50000000
SERVICE_DISCIPLINE = 'FCFS'
LOGGED = True
VERBOSE = False

### Job

In [3]:
class Job:
    def __init__(self, name, arrtime, duration):
        self.name = name
        self.arrtime = arrtime
        self.duration = duration
        
    def __str__(self):
        return '%s at %d, length %d' % (self.name, self.arrtime, self.duration)

### Server

class Server:
    def __init__(self, env, strat, job, servernum):
        self.env = env
        self.strat = strat
        self.servernum = servernum
        self.job = job
        self.serversleeping = None
        ''' statistics '''
        self.waitingTime = 0
        self.idleTime = 0
        self.jobsDone = 0
        ''' register a new server process '''
        env.process( self.serve() )
        
    def serve(self):
        ''' do nothing, just change server to idle
            and then yield a wait event which takes infinite time
        '''
        while True:
            if self.job == None:
                self.serversleeping = env.process( self.waiting( self.env ))
                t1 = self.env.now
                yield self.serversleeping
                ''' accumulate the server idle time'''
                self.idleTime += self.env.now - t1

            else:
                if LOGGED:
                    qlog[i].write('%.4f\t%d\n' % (self.env.now, 1 if self.job != None else 0))

                ''' sum up the waiting time'''
                self.waitingTime += self.env.now - j.arrtime
                ''' yield an event for the job finish'''
                yield self.env.timeout( j.duration )
                ''' sum up the jobs done '''
                self.jobsDone += 1
                self.job = None
                
    def waiting(self, env):
        try:
            if VERBOSE:
                print( 'Server is idle at %.2f' % self.env.now )
            yield self.env.timeout( MAXSIMTIME )
        except simpy.Interrupt as i:
            if VERBOSE:
                 print('Server waken up and works at %.2f' % self.env.now )

### System

In [4]:
class System:
    def __init__(self, env, servernum, strat):
        self.env = env
        self.strat = strat
        self.servernum = servernum
        self.systemsleeping = None
        self.Jobs = list(())
        env.process( self.start() )
        self.Servers = [None] * servernum
        for i in range(0, servernum):
            self.Servers[i] = self.Server(env, self.strat, None, i)
            
    def start(self):
        while True:
            if len(self.Jobs) == 0:
                self.systemsleeping = env.process(self.waiting(self.env))
                yield self.systemsleeping
            else:
                for i in range(0, self.servernum):
                    if len(self.Jobs) == 0: continue
                    if self.Servers[i].job == None:
                        self.Servers[i].job = self.Jobs.pop(0)
                        slog.write('%.4f\t%d\t%d\n' % (self.env.now, i, len(self.Jobs)))
                
                
    def waiting(self, env):
        try:
            if VERBOSE:
                print( 'System is idle at %.2f' % self.env.now )
            yield self.env.timeout( MAXSIMTIME )
        except simpy.Interrupt as i:
            if VERBOSE:
                 print('Server waken up and works at %.2f' % self.env.now )
        
    class Server:
        def __init__(self, env, strat, job, servernum):
            self.env = env
            self.strat = strat
            self.servernum = servernum
            self.job = job
            self.serversleeping = None
            ''' statistics '''
            self.waitingTime = 0
            self.idleTime = 0
            self.jobsDone = 0
            ''' register a new server process '''
            env.process( self.serve() )

        def serve(self):
            ''' do nothing, just change server to idle
                and then yield a wait event which takes infinite time
            '''
            while True:
                if self.job == None:
                    self.serversleeping = env.process( self.waiting( self.env ))
                    t1 = self.env.now
                    yield self.serversleeping
                    ''' accumulate the server idle time'''
                    self.idleTime += self.env.now - t1

                else:
                    if LOGGED:
                        qlog[self.servernum].write('%.4f\t%d\n' % (self.env.now, 1 if self.job != None else 0))

                    ''' sum up the waiting time'''
                    self.waitingTime += self.env.now - self.job.arrtime
                    ''' yield an event for the job finish'''
                    yield self.env.timeout( self.job.duration )
                    ''' sum up the jobs done '''
                    self.jobsDone += 1
                    self.job = None

        def waiting(self, env):
            try:
                if VERBOSE:
                    print( 'Server is idle at %.2f' % self.env.now )
                yield self.env.timeout( MAXSIMTIME )
            except simpy.Interrupt as i:
                if VERBOSE:
                     print('Server waken up and works at %.2f' % self.env.now )
        
    
    

### Job generator

In [5]:
class JobGenerator:
    def __init__(self, env, system, nrjobs, lam, mu):
        self.system = system
        self.nrjobs = nrjobs
        self.interarrivaltime = 1/lam;
        self.servicetime = 1/mu;
        env.process( self.generatejobs(env) )
        
    def generatejobs(self, env):
        i = 1
        while True:
            '''yield an event for new job arrival'''
            job_interarrival = random.exponential( self.interarrivaltime )
            yield env.timeout( job_interarrival )

            ''' generate service time and add job to the list'''
            job_duration = random.exponential( self.servicetime )
            self.system.Jobs.append( Job('Job %s' %i, env.now, job_duration) )
            if VERBOSE:
                print( 'job %d: t = %.2f, l = %.2f, dt = %.2f' 
                    %( i, env.now, job_duration, job_interarrival ) )
            i += 1

            ''' if server is idle, wake it up'''
            if not self.system.systemsleeping.triggered:
                self.system.systemsleeping.interrupt( 'Wake up, please.' )
                for i in range(0, c):
                    if not self.system.Servers[i].serversleeping.triggered:
                        self.system.Servers[i].serversleeping.interrupt('Wake up, please')

### Log file

In [6]:
qlog = [None] * c
if LOGGED:
    slog = open( 'mmc-l%d-m%d-c%d.csv' % (LAMBDA,MU,c), 'w')
    slog.write('0\t0\n')
    for i in range(0, c):
        qlog[i] = open( 'mmc-l%d-m%d-%d-%d-example.csv' % (LAMBDA,MU,i+1,c), 'w' )
        qlog[i].write( '0\t0\n' )

### Start Simpy

In [7]:
env = simpy.Environment()
MySystem = System(env, c, SERVICE_DISCIPLINE)
MyJobGenerator = JobGenerator(env, MySystem, POPULATION, LAMBDA, MU)

### Run the simulation

In [8]:
env.run( until = MAXSIMTIME )

Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "C:\Users\BIN\AppData\Local\Temp\ipykernel_12032\1770112328.py", line -1, in start
KeyboardInterrupt

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "C:\Users\BIN\AppData\Local\Programs\Python\Python310\lib\site-packages\IPython\core\interactiveshell.py", line 3433, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "C:\Users\BIN\AppData\Local\Temp\ipykernel_12032\1650718486.py", line 1, in <module>
    env.run( until = MAXSIMTIME )
  File "C:\Users\BIN\AppData\Local\Programs\Python\Python310\lib\site-packages\simpy\core.py", line 254, in run
    self.step()
  File "C:\Users\BIN\AppData\Local\Programs\Python\Python310\lib\site-packages\simpy\core.py", line 206, in step
    raise exc
KeyboardInterrupt

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\BIN\AppData\Local\Programs\Python\Pytho

### Close the log file

In [9]:
if LOGGED:
    slog.close()
    for i in range(0, c):
        qlog[i].close()